In [5]:
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, summarize_and_visualize

ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
START_DATE = "2020-12-19 08:00"
END_DATE = "2024-12-19 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE"}
transfers_df = fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=10000,
    )
# final_record_df  = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, BASE_TOKENS)
# summary_df = summarize_and_visualize(final_record_df, BASE_TOKENS)

2024-12-23 14:34:13,940 - INFO - Start block: 11480310, End block: 21432853
2024-12-23 14:34:13,940 - INFO - Fetching ERC20 transfers from block 11480310 to 21432853...
2024-12-23 14:34:17,345 - INFO - Page 1: Retrieved 4673 transactions.
2024-12-23 14:34:18,261 - WARNING - API request failed on page 2: Request failed: Result window is too large, PageNo x Offset size must be less than or equal to 10000
2024-12-23 14:34:18,261 - INFO - Final dataset contains 4673 unique transactions.
2024-12-23 14:34:18,301 - INFO - Data successfully saved to C:/Users/YuweiCao/Documents/GitHub/Project/etherscan/result\erc20_transfers.csv
2024-12-23 14:34:18,302 - INFO - Transfers saved to C:/Users/YuweiCao/Documents/GitHub/Project/etherscan/result
